In [ ]:
api_key = ""

In [ ]:
import os
import json
import requests
form_id = ""

## fetch form properties

In [ ]:
headers = {
    'apiKey': api_key
}
url = f"https://api.jotform.com/form/{form_id}/properties"
response = requests.request("GET", url, headers=headers)

In [25]:
form_properties = json.loads(response.text)

In [ ]:
form_properties

In [14]:
with open("form_properties.json", mode="w", encoding="utf-8") as write_file:
    json.dump(form_properties, write_file)

## fetch form questions

In [ ]:
url = f"https://api.jotform.com/form/{form_id}/questions"
response = requests.request("GET", url, headers=headers)

In [16]:
form_questions = json.loads(response.text)

In [17]:
with open("form_questions.json", mode="w", encoding="utf-8") as write_file:
    json.dump(form_questions, write_file)

## Get the enabled conditions

In [ ]:
conditions = form_properties["content"]["conditions"]
active_conditions = []
for condition in conditions:
    if "disabled" in condition and condition["disabled"] == "1":
        continue
    if condition["type"] == "field" or condition["type"] == "page":
        active_conditions.append(condition)

In [ ]:
active_conditions

In [19]:
condition_types = {
    "HideMultiple": "hide",
    "Hide": "hide",
    "ShowMultiple": "show",
    "Show": "show",
    "hidePage": "hide page",
    "skipTo": "skip page"
}

In [ ]:
result_fields = {}
def add_result_field(field, triggers, action):
    print(f"action:{action}")
    print(f"result_fields:{result_fields}")
    print(f"field:{field}")
    print(f"trigger:{triggers}")
    action_type = condition_types[action]
    action = condition_types[action]
    if field not in result_fields:
        result_fields[field] = {}
    if action not in result_fields[field]:
        result_fields[field][action] = []
    for trigger in triggers:
        if trigger not in result_fields[field][action]:
            result_fields[field][action].append(trigger)
    print(f"result_fields:{result_fields}")

In [ ]:
result_fields

In [ ]:
active_conditions

In [ ]:
for condition in active_conditions:
    print(condition)
    actions = json.loads(condition["action"])
    if True:
        print("================================")
        print("================================")
        terms = json.loads(condition["terms"].replace("\\\xa0", " "))
        if len(terms) > 1:
            print(f"IF {condition["link"]}:")
        else:
            print("IF:")
        triggers = []
        trigger_list_all = []
        for term in terms:
            operator = term["operator"]
            field_id = term["field"]
            value = term["value"]
            if condition["link"] == "All":
                trigger_list_all.append(f"{field_id} {operator} {value}")
            else:
                triggers.append([f"{field_id} {operator} {value}"])
            print(f" - IF field {field_id} is {operator} to {value}")
        if trigger_list_all:
            triggers.append(trigger_list_all)
        print("\nTHEN:")
        actions = json.loads(condition["action"])
        for action in actions:
            if "visibility" in action:
                action_type = action["visibility"]
                if "fields" in action:
                    for field in action["fields"]:
                        print(f" - {condition_types[action_type]} {field}")
                        add_result_field(field, triggers, action_type)
                else:
                    field = action["field"]
                    add_result_field(field, triggers, action_type)
                    print(f" - {condition_types[action_type]} {field}")
            if "skipHide" in action:
                page = action["skipTo"]
                action_type = action["skipHide"]
                add_result_field(page, triggers, action_type)
                print("this is after add result?")
                print(f" - {condition_types[action_type]} {page}")

In [ ]:
def field_name(id):
    if id in form_questions["content"]:
        name = form_questions["content"][id]["text"]
        order = form_questions["content"][id]["order"]
        return f"{order}. {name}"
    else:
        return id

def get_trigger_text(text):
    id = text.split(" ")[0]
    field_text = field_name(id)
    condition_text = text[text.find(" "):]
    return field_text, condition_text

In [ ]:
result_fields

{'134': {'show': [['6 equals 67'],
   ['6 equals 66'],
   ['6 equals 48'],
   ['6 equals 47'],
   ['6 equals 44'],
   ['6 equals 43'],
   ['6 equals 42'],
   ['6 equals 41'],
   ['6 equals 40'],
   ['6 equals 39'],
   ['6 equals 38'],
   ['6 equals 37'],
   ['6 equals 36'],
   ['6 equals 35'],
   ['6 equals 34'],
   ['6 equals 31']]},
 '135': {'show': [['6 equals 67'],
   ['6 equals 66'],
   ['6 equals 48'],
   ['6 equals 47'],
   ['6 equals 44'],
   ['6 equals 43'],
   ['6 equals 42'],
   ['6 equals 41'],
   ['6 equals 40'],
   ['6 equals 39'],
   ['6 equals 38'],
   ['6 equals 37'],
   ['6 equals 36'],
   ['6 equals 35'],
   ['6 equals 34'],
   ['6 equals 31']]},
 '136': {'show': [['6 equals 67'],
   ['6 equals 66'],
   ['6 equals 48'],
   ['6 equals 47'],
   ['6 equals 44'],
   ['6 equals 43'],
   ['6 equals 42'],
   ['6 equals 41'],
   ['6 equals 40'],
   ['6 equals 39'],
   ['6 equals 38'],
   ['6 equals 37'],
   ['6 equals 36'],
   ['6 equals 35'],
   ['6 equals 34'],
   ['6 equa

In [ ]:
for field_id in result_fields:
    field = result_fields[field_id]
    print()
    for action_type in field:
        print("IF:")
        for trigger_set in field[action_type]:
            if len(trigger_set) > 1:
                print("  IF ALL:")
                for trigger in trigger_set:
                    field_text, condition_text = get_trigger_text(trigger)
                    print(f"    {field_text}")
                    print(f"      {condition_text}")
            else:
                field_text, condition_text = get_trigger_text(trigger_set[0])
                print(f"  {field_text}")
                print(f"    {condition_text}")
        print(f"{action_type} {field_name(field_id)}")


IF:
  6. Lesson Number
     equals 67
  6. Lesson Number
     equals 66
  6. Lesson Number
     equals 48
  6. Lesson Number
     equals 47
  6. Lesson Number
     equals 44
  6. Lesson Number
     equals 43
  6. Lesson Number
     equals 42
  6. Lesson Number
     equals 41
  6. Lesson Number
     equals 40
  6. Lesson Number
     equals 39
  6. Lesson Number
     equals 38
  6. Lesson Number
     equals 37
  6. Lesson Number
     equals 36
  6. Lesson Number
     equals 35
  6. Lesson Number
     equals 34
  6. Lesson Number
     equals 31
show 15. Pilot Qualifications

IF:
  6. Lesson Number
     equals 67
  6. Lesson Number
     equals 66
  6. Lesson Number
     equals 48
  6. Lesson Number
     equals 47
  6. Lesson Number
     equals 44
  6. Lesson Number
     equals 43
  6. Lesson Number
     equals 42
  6. Lesson Number
     equals 41
  6. Lesson Number
     equals 40
  6. Lesson Number
     equals 39
  6. Lesson Number
     equals 38
  6. Lesson Number
     equals 37
  6. Les